## Cleaning and Normalization

In [12]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)
# loading saved objects into dataframe
#df = pd.read_pickle('JC_residential.pkl')
#errors_df = pd.read_pickle('errors_df.pkl')

#jc_deed_df = pd.read_csv('0906130957_deed.csv', index_col=False)

df = pd.read_csv('0906monm220028.csv', usecols=['propertyLocation', 'ownersName', 'ownersMailingAddress', 'cityStateZip'])

# number of times owner appears in second column
def propertiesOwned(owner=str):
    return len(df[df.ownersName == owner])

# number of times property appears in first column
def propertyAppearances(address=str): 
    return len(df[df.propertyLocation == address])

df['ownersFullMailingAddress'] = df['ownersMailingAddress'] + ', ' + df.cityStateZip
df['propertyFullAddress'] = df.propertyLocation + ' Jersey City NJ'


In [14]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress
0,1075 SECAUCUS RD.,"20 AQUARIUM DR.,LLC%NAT'L RETAIL",611 ROUTE 46 WEST,"HASBROUCK HEIGHTS, N.J. 07604","611 ROUTE 46 WEST, HASBROUCK HEIGHTS, N.J. 07604",1075 SECAUCUS RD. Jersey City NJ
1,1045 SECAUCUS RD.,PUBLIC SERVICE ELECTRIC & GAS CO.,80 PARK PL.TAX DEPT.T-6B,"NEWARK, N.J. 07102","80 PARK PL.TAX DEPT.T-6B, NEWARK, N.J. 07102",1045 SECAUCUS RD. Jersey City NJ
2,SECAUCUS ROAD,UNITED STATES OF AMERICA,"475 L'ENFANT PLAZA, SW","WASHINGTON, D C 20260","475 L'ENFANT PLAZA, SW, WASHINGTON, D C 20260",SECAUCUS ROAD Jersey City NJ
3,INSIDE SECAUCUS RD.,HOLLAND CO.C/O PUB.SER.GAS&ELEC.CO.,80 PARK PL.-TAX DEPT.T-6B,"NEWARK, N.J. 07102","80 PARK PL.-TAX DEPT.T-6B, NEWARK, N.J. 07102",INSIDE SECAUCUS RD. Jersey City NJ
4,SECAUCUS RD,UNITED STATES OF AMERICA,"475 L'ENFANT PLAZA, SW","WASHINGTON, D C 20260","475 L'ENFANT PLAZA, SW, WASHINGTON, D C 20260",SECAUCUS RD Jersey City NJ
...,...,...,...,...,...,...
63829,VARIOUS,VERIZON-NJ C/O DUFF & PHELPS,P.O. BOX 2749,"ADDISON, TX 75001","P.O. BOX 2749, ADDISON, TX 75001",VARIOUS Jersey City NJ
63830,VARIOUS,"TEXAS EASTERN TRANSMISSION,LP",P.O. BOX 2629,"ADDISON, TX 75001","P.O. BOX 2629, ADDISON, TX 75001",VARIOUS Jersey City NJ
63831,VARIOUS,"AIRBNB, INC",888 BRANNAN ST.,"SAN FRANCISCO, CA 94103","888 BRANNAN ST., SAN FRANCISCO, CA 94103",VARIOUS Jersey City NJ
63832,VARIOUS,TRANSCONTINENTAL GAS PIPE LINE,"P.O. BOX 2400, MD 46-4","TULSA, OK 74102","P.O. BOX 2400, MD 46-4, TULSA, OK 74102",VARIOUS Jersey City NJ


In [2]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [3]:
len(geocode_errors) == len(errors_df)

True

In [4]:
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
2529,78A RNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A RNE ST. Jersey City, NJ",1,1,None
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [30]:
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TONNELE', 'TONNELLE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC ADOO', 'MCADOO')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DRIVE', 'MARTIN LUTHER KING DRIVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COLUMBUS', 'CHRISTOPHER COLUMBUS')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIRST', '1st')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SECOND', '2nd')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THIRD', '3rd')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOURTH', '4th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTH', '5th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTH', '6th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVENTH', '7th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EIGHTH', '8th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINETH', '9th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TENTH', '10th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINTH', '9th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTEENTH', '15th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTEENTH', '16th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC DOUGALL', 'MCDOUGALL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('CARPENTIER', 'CARPENTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOX HOUND', 'FOXHOUND')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIEDLER', 'SEIDLER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LIENAU', 'LINEAU')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('KENNEDY', 'JOHN F KENNEDY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LIBERTY ST.', 'LIBERTY AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-REAR', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('(REAR)', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-FRONT', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HAMPTON CT.', 'HAMPTON COURT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VARCK', 'VARICK')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THO', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE AVE', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TER', 'TR')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3633A', '3633')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3144A', '3144')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SHEARWATR', 'SHEARWATER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PALUSL', "PAUL'S")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COURT TR', "CT")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EASTVIEW', "E VIEW")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DR.', 'MARTIN LUTHER KING DR.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 1ST FL.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' SO.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' N.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ODGEN', 'OGDEN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PATRSON', 'PATERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 61 COLES', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VAN HOUTEN ST.', 'VAN HOUTEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON ST.', 'MCPHERSON PL.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON', 'MCPHERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIR ROAD', 'FIR ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('UNION AVE.', 'UNION ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('OLD BERGEN', 'OLD BERGEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE. AVE.', 'AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('JACKSON AVE.', 'JACKSON ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('162 OLD BERGEN AVE. AVE.', '162 OLD BERGEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLEN', 'GLENN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLENNN', 'GLENN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('DR.VE', 'DRIVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('1 GLENNN LANE', '1 GLENN LANE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\(\)', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}(.?\d{1}(A)?)?A?\b', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\s#\d', '')


# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BUTTRNUT', 'BUTTERNUT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('CARPENTR', 'CARPENTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('100 SHEARWATER CT.', '100 EAST SHEARWATER CT.')

errors_df.propertyLocation = errors_df.propertyLocation.str.replace('78A RNE ST.', '78 THORNE ST.')

#overwriting past mistakes
errors_df.propertyFullAddress = errors_df.propertyLocation + ' Jersey City, NJ'

/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_1440/1358142991.py:69: FutureWarning: The default value of regex will change from True to False in a future version.
  errors_df.propertyLocation = errors_df.propertyLocation.str.replace('78A RNE ST.', '78 THORNE ST.')


In [31]:
%%time
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

CPU times: user 21.6 ms, sys: 3.5 ms, total: 25.1 ms
Wall time: 2.91 s


In [32]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [33]:
errors_df.iloc[geocode_errors].propertyLocation.unique()

array(['10 PAULMIER PL.', '280 GREGORY PARK PLAZA',
       '270 GREGORY PARK PLAZA', '32 WILKINSON ST.', '107 SHEARWATER CT.'],
      dtype=object)

In [34]:
len(errors_df.iloc[geocode_errors].propertyLocation.unique())

5

### Before we save the smaller errors data frame, we need to append the cleaned errors to the original df

In [35]:
df.append(errors_df.dropna(axis=0, subset=['gCode'])).to_pickle('JC_residential.pkl')

In [36]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [37]:
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [38]:
# reassigning the errors_df to the remaining errors
errors_df = errors_df.iloc[geocode_errors].copy()

In [39]:
errors_df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [40]:
errors_df.to_pickle('errors_df.pkl')

In [41]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode,latitude,longitude
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307","677 LIBERTY AVE. Jersey City, NJ",1,1,"(677, Liberty Avenue, Jersey City, Hudson Coun...",40.759744,-74.050457
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307","675 LIBERTY AVE. Jersey City, NJ",1,1,"(675, Liberty Avenue, Jersey City, Hudson Coun...",40.759679,-74.050499
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307","673 LIBERTY AVE. Jersey City, NJ",1,1,"(673, Liberty Avenue, Jersey City, Hudson Coun...",40.759614,-74.050542
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307","671 LIBERTY AVE. Jersey City, NJ",1,1,"(671, Liberty Avenue, Jersey City, Hudson Coun...",40.759549,-74.050584
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307","669 LIBERTY AVE. Jersey City, NJ",1,1,"(669, Liberty Avenue, Jersey City, Hudson Coun...",40.759484,-74.050626
...,...,...,...,...,...,...,...,...,...,...,...
38273,100 EAST SHEARWATER CT.,"BALANLAYOS, JESUS & MYRNA",31 HARMONY WAY,"SEWAREN, NJ 07077","31 HARMONY WAY, SEWAREN, NJ 07077","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38276,100 EAST SHEARWATER CT.,"DANGCIL, PROCERFINA & WILLIAM A JR",100-51 E SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-51 E SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38278,100 EAST SHEARWATER CT.,"LAWSONS REALTY,LLC",130 SHORE RD #198,"PORT WASHINGTON, NY 11050","130 SHORE RD #198, PORT WASHINGTON, NY 11050","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38279,100 EAST SHEARWATER CT.,"NATHAN, NAVIN D.",100-54 SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-54 SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935


In [42]:
df['latitude'] = [g.latitude for g in df.gCode]
df['longitude'] = [g.longitude for g in df.gCode]

In [43]:
df.append(errors_df.dropna(axis=0, subset=['gCode'])).to_pickle('JC_residential.pkl')

In [44]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode,latitude,longitude
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307","677 LIBERTY AVE. Jersey City, NJ",1,1,"(677, Liberty Avenue, Jersey City, Hudson Coun...",40.759744,-74.050457
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307","675 LIBERTY AVE. Jersey City, NJ",1,1,"(675, Liberty Avenue, Jersey City, Hudson Coun...",40.759679,-74.050499
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307","673 LIBERTY AVE. Jersey City, NJ",1,1,"(673, Liberty Avenue, Jersey City, Hudson Coun...",40.759614,-74.050542
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307","671 LIBERTY AVE. Jersey City, NJ",1,1,"(671, Liberty Avenue, Jersey City, Hudson Coun...",40.759549,-74.050584
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307","669 LIBERTY AVE. Jersey City, NJ",1,1,"(669, Liberty Avenue, Jersey City, Hudson Coun...",40.759484,-74.050626
...,...,...,...,...,...,...,...,...,...,...,...
38273,100 EAST SHEARWATER CT.,"BALANLAYOS, JESUS & MYRNA",31 HARMONY WAY,"SEWAREN, NJ 07077","31 HARMONY WAY, SEWAREN, NJ 07077","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38276,100 EAST SHEARWATER CT.,"DANGCIL, PROCERFINA & WILLIAM A JR",100-51 E SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-51 E SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38278,100 EAST SHEARWATER CT.,"LAWSONS REALTY,LLC",130 SHORE RD #198,"PORT WASHINGTON, NY 11050","130 SHORE RD #198, PORT WASHINGTON, NY 11050","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38279,100 EAST SHEARWATER CT.,"NATHAN, NAVIN D.",100-54 SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-54 SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935


In [19]:
df.to_csv('JC_residential.csv')